In [ ]:
#！/usr/bin/python
# -*- coding: utf-8 -*-

In [ ]:
import pyodbc
import textwrap
import pandas as pd

import plotly
import plotly.plotly as py
import plotly.graph_objs as go

In [ ]:
sql1 = textwrap.dedent("""
    IF OBJECT_ID('tempdb..#Diversity') IS NOT NULL
        DROP TABLE #Diversity
""")
sql2 = textwrap.dedent("""
    SELECT userid
          ,date
          ,COUNT(DISTINCT website) AS cs
    INTO #Diversity
    FROM [data].[dbo].[health_records]
    GROUP BY userid, date
    ORDER BY userid, date
""")
sql3 = textwrap.dedent("""
    SELECT cs
          ,COUNT(cs) AS sessions
          ,CONVERT(DECIMAL(4,4), COUNT(cs)/CONVERT(DECIMAL(6,2), (SELECT COUNT(cs) FROM #Diversity))) AS pct
    FROM #Diversity
    GROUP BY cs
    ORDER BY cs
""")

In [ ]:
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=localhost;DATABASE=data;')

cursor = cnxn.cursor()

In [ ]:
cursor.execute(sql1)
cursor.execute(sql2)
rows=cursor.execute(sql3).fetchall()

In [ ]:
# 加载查询结果并对列命名
df = pd.DataFrame( [[ij for ij in i] for i in rows] )
df.rename(columns={0:'Consideration Set', 1: 'Sessions', 2:'Percentage'}, inplace=True)

In [ ]:
# 访问多个网站的情况
dfx = df.loc[1:]
# 重新计算分布
dfx['Percentage'] = dfx.Sessions/dfx.Sessions.sum()
dfx

In [ ]:
# 计算访问2或3个网站的session的占比
dfx.loc[1:2].Percentage.sum()

In [ ]:
# 计算平均consideration set
t = df['Consideration Set']*df['Percentage']
t.sum()

In [ ]:
# 计算平均consideration set（除去只访问1个网站的情况）
t = dfx['Consideration Set']*dfx['Percentage']
t.sum()

In [ ]:
# 添加数据，使图更美观
df.loc[6] = [7,0,0]
df

In [ ]:
plotly.tools.set_credentials_file(username='tenaghirmb', api_key='please_use_your_own_key')

plotly.offline.init_notebook_mode(connected=True)

In [ ]:
trace = go.Bar(
    x = df['Consideration Set'],
    y = df['Percentage'],
    text = df['Percentage'].apply(lambda x:format(x,'.2%')),
    textposition = 'outside',
    textfont = dict(size=9,),
    width = [0.35]*7
)

data = [trace]

layout = go.Layout(
    xaxis=dict(
        title='Number of websites accessed',
        titlefont=dict(
            size=16,
        ),
        ticklen=2,
        tickfont=dict(
            size=12,
        ),
        nticks=10,
    ),
    yaxis=dict(
        title='Percentage of Sessions',
        titlefont=dict(
            size=16,
        ),
        tickfont=dict(
            size=11,
        ),
        tickformat='.0%',
        nticks=15,
        #range=[0,1],
    ),
)

fig = go.Figure(data=data, layout=layout)

plotly.offline.iplot(fig, filename='Consideration Set each Session', image='png')

#py.iplot(fig, filename='Consideration Set each Session')